In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Import data
athletes = pd.read_csv("data/athletes.csv")
counts = pd.read_csv("data/counts.csv")

# This dataframe will be the final output
# Initialize as all columns of counts except 'Rank'
olympics = counts.drop(columns=['Rank'])

# Order by 'NOC' and 'Year'
olympics = olympics.sort_values(by=['NOC', 'Year'])
olympics = olympics[olympics['Year'] >= 1944]

In [3]:
# Feature 1: Total number of athletes per NOC and Year
olympics = olympics.merge(
    athletes.groupby(['NOC', 'Year']).size().reset_index(name='X.Athletes'),
    on=['NOC', 'Year'],
    how='left'
)

In [4]:
# Feature 2: Total number of events per NOC and Year
olympics = olympics.merge(
    athletes.groupby(['NOC', 'Year'])['Event'].nunique().reset_index(name='X.Events'),
    on=['NOC', 'Year'],
    how='left'
)

In [5]:
# Feature 3: Dummy variable for host country
# The year when the selected NOC hosted the Olympics
# are manually constructed from the data summerOly_host.csv

host_years = {
    'USA': [1904, 1932, 1984, 1996, 2028],
    'CHN': [2008],
    'JPN': [1964, 2020],
    'AUS': [1956, 2000],
    'FRA': [1900, 1924, 1968, 1992, 2024],
}

# Create the column 'isHost'
olympics['IsHost'] = olympics.apply(
    lambda row: 1 if row['Year'] in host_years[row['NOC']] else 0,
    axis=1
)

In [6]:
# Add country index
olympics["Country_id"] = olympics["NOC"].map({
    'USA': 1,
    'CHN': 2,
    'JPN': 3,
    'AUS': 4,
    'FRA': 5,
})

In [7]:
# Add year index
olympics["Year_id"] = olympics["Year"].apply(
    lambda x: 1 + (int(x)-1896)/4
)

In [8]:
# Save the final data
olympics.to_csv("data/olympics.csv", index=False)

In [24]:
# Trends in medal counts and number of athletes
fig, axes = plt.subplots(
    nrows=5, ncols=2,
    figsize=(7.0, 10.0),
    constrained_layout=True,
    sharex=False
)

countries = ["USA", "CHN", "JPN", "AUS", "FRA"]

for r, noc in enumerate(countries):
    data = olympics[olympics["NOC"] == noc]

    ax_m = axes[r, 0]
    ax_m.plot(data["Year"], data["Gold"],   marker="o", label="Gold")
    ax_m.plot(data["Year"], data["Silver"], marker="o", label="Silver")
    ax_m.plot(data["Year"], data["Bronze"], marker="o", label="Bronze")
    ax_m.set_title(f"{noc} - Medal Counts", fontsize=12, fontweight="bold")
    ax_m.set_xlabel("Year")
    ax_m.set_ylabel("Count")
    ax_m.legend(frameon=False, ncol=3, loc="upper left")

    ax_a = axes[r, 1]
    ax_a.plot(data["Year"], data["X.Athletes"],
              marker="o", color="tab:blue")
    ax_a.set_title(f"{noc} - Number of Athletes", fontsize=12, fontweight="bold")
    ax_a.set_xlabel("Year")
    ax_a.set_ylabel("Athletes")

fig.savefig("figures/trend.pdf",
            format="pdf",
            bbox_inches="tight")

plt.close(fig)